Pollenate TO Dataset

In [1]:
import geopandas as gpd
import pandas as pd
import os
import folium
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint

In [2]:
# Path to your GeoJSON file
geojson_file_path = 'PollinateTO Primary Project Garden Locations - 4326.geojson'

# Read the GeoJSON file
pollinateto_data = gpd.read_file(geojson_file_path)


In [3]:
# Display the first few rows
pollinateto_data.head()

,_id,ID,YEAR_FUNDED,GROUP_NAME,PROJECT_NAME,GARDEN_TYPE,NUMBER_OF_GARDENS,ESTIMATED_GARDEN_SIZE,NEIGHBOURHOOD_NUMBER,NIA_OR_EN,PRIMARY_LOCATION_NAME,PRIMARY_GARDEN_ADDRESS,POSTAL_CODE,IS_INDIGENOUS_GARDEN,WARD_NAME,WARD_NUMBER,DESCRIPTION,IMAGE_NAME,IMAGE_ALT_TEXT,geometry
0,1,1,2020,Appleton Ave. Community Organization,Appleton Ave. Pollinator Corridor,Multiple Front Yard Gardens,18,1250.00,92,None,Private Residential Property,14 Appleton Avenue,M6E 3A4,None,Davenport,9,None,None,None,MULTIPOINT (-79.43796 43.67956)
1,2,2,2020,Centre for Immigrant and Community Services (C...,The ENRICH Pollinator Garden,Communal Garden,4,114.25,128,None,Centre for Immigrant and Community Services,2330 Midland Avenue,M1S 5G5,None,Scarborough-Agincourt,22,None,None,None,MULTIPOINT (-79.27601 43.77839)
2,3,3,2020,Chester School Pollinator Garden Team,Chester School Pollinator Garden,School Learning Garden,1,34.00,57,None,Chester Elementary School,115 Gowan Avenue,M4K 2E4,None,Toronto-Danforth,14,None,None,None,MULTIPOINT (-79.35171 43.6876)
3,4,4,2020,CSPC - Transfiguration,Transfiguration Pollinator Garden,School Learning Garden,1,22.00,7,None,Transfiguration of Our Lord Catholic School,55 Ludstone Drive,M9R 2J2,None,Etobicoke Centre,2,None,None,None,MULTIPOINT (-79.56287 43.69153)
4,5,5,2020,Danforth Gardens Neighbourhood Association,Danforth Gardens Neighbourhood Pollinator Corr...,Multiple Front Yard Gardens,7,28.00,120,None,Private Residential Property,64 Clapperton Avenue,M1L 4K7,None,Scarborough Southwest,20,None,None,None,MULTIPOINT (-79.27635 43.71211)


In [4]:
len(pollinateto_data)

149

In [5]:
years = pollinateto_data['YEAR_FUNDED'].unique()
print(years)

[2020 2021 2022 2023]


In [6]:
print("Columns with null values:\n")
print(pollinateto_data.isnull().sum()[pollinateto_data.isnull().sum() > 0])

# If no null values are found
if pollinateto_data.isnull().sum().sum() == 0:
    print("\nNo null values found in the dataset.")

Columns with null values:

Series([], dtype: int64)

No null values found in the dataset.


In [7]:
pollinateto_data['WARD_NAME'].value_counts()

WARD_NAME
Parkdale-High Park          16
Toronto-Danforth            13
Toronto Centre              11
Davenport                   10
Beaches-East York           10
University-Rosedale          9
Scarborough Southwest        9
Scarborough-Guildwood        7
Etobicoke-Lakeshore          6
Toronto-St. Paul's           6
Humber River-Black Creek     6
Don Valley East              5
Don Valley West              5
Scarborough-Agincourt        5
York South-Weston            4
Spadina-Fort York            4
Scarborough-Rouge Park       4
Etobicoke Centre             4
York Centre                  3
Eglinton-Lawrence            3
Etobicoke North              3
Etobicoke-North              2
Willowdale                   2
Scarborough Centre           1
Don Valley North             1
Name: count, dtype: int64

Filter Private and Public Gardens

In [8]:
private_gardens = pollinateto_data[pollinateto_data['PRIMARY_LOCATION_NAME'].str.contains('Private', case=False, na=False)]

school_gardens = pollinateto_data[pollinateto_data['GARDEN_TYPE'].str.contains('School', case=False, na=False)]

# The rest go to 'public_gardens'
public_gardens = pollinateto_data[~pollinateto_data['PRIMARY_LOCATION_NAME'].str.contains('Private', case=False, na=False)]

school_gardens

,_id,ID,YEAR_FUNDED,GROUP_NAME,PROJECT_NAME,GARDEN_TYPE,NUMBER_OF_GARDENS,ESTIMATED_GARDEN_SIZE,NEIGHBOURHOOD_NUMBER,NIA_OR_EN,PRIMARY_LOCATION_NAME,PRIMARY_GARDEN_ADDRESS,POSTAL_CODE,IS_INDIGENOUS_GARDEN,WARD_NAME,WARD_NUMBER,DESCRIPTION,IMAGE_NAME,IMAGE_ALT_TEXT,geometry
2,3,3,2020,Chester School Pollinator Garden Team,Chester School Pollinator Garden,School Learning Garden,1,34.0,57,None,Chester Elementary School,115 Gowan Avenue,M4K 2E4,None,Toronto-Danforth,14,None,None,None,MULTIPOINT (-79.35171 43.6876)
3,4,4,2020,CSPC - Transfiguration,Transfiguration Pollinator Garden,School Learning Garden,1,22.0,7,None,Transfiguration of Our Lord Catholic School,55 Ludstone Drive,M9R 2J2,None,Etobicoke Centre,2,None,None,None,MULTIPOINT (-79.56287 43.69153)
5,6,6,2020,David Hornell Junior School & Mimico Residents...,David Hornell Pollinator Gardens,School Learning Garden,3,111.0,161,None,David Hornell Junior Public School,32 Victoria Street,M8V 1M6,None,Etobicoke-Lakeshore,3,None,None,None,MULTIPOINT (-79.48889 43.61933)
7,8,8,2020,FJR Pollinator Project,FJR Pollinator Garden,School Learning Garden,3,46.5,18,None,Father John Redmond Catholic Secondary School ...,28 Colonel Samuel Smith Park Drive,M8V 4B6,None,Etobicoke-Lakeshore,3,None,None,None,MULTIPOINT (-79.51654 43.59514)
10,11,11,2020,Green Thumbs Growing Kids,Flower Power,School Learning Garden,3,5260.0,74,None,Rose Avenue Junior Public School,675 Ontario Street,M4X 1N4,None,Toronto Centre,13,None,None,None,MULTIPOINT (-79.37279 43.67032)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,145,145,2023,Warren Park JPS Parents and Teachers,Warren Park Pollinator Garden,School Learning Garden,3,37.0,114,None,Warren Park Junior Public School,135 Varsity Road,M6S 4P4,None,Parkdale-High Park,4,None,None,None,MULTIPOINT (-79.49944 43.66285)
145,146,146,2023,West Hill DD Program,Pollinator Gardens,School Learning Garden,1,11.0,136,NIA,West Hill Collegiate Institute,350 Morningside Avenue,M1E 3G3,None,Scarborough-Guildwood,24,None,None,None,MULTIPOINT (-79.19059 43.77567)
146,147,147,2023,Westmount JS Pollinator Club,Westmount Pollinator Garden,School Learning Garden,1,10.0,8,EN,Westmount Junior School,95 Chapman Road,M9P 1E9,Y,Etobicoke Centre,2,None,None,None,MULTIPOINT (-79.51966 43.6891)
147,148,148,2023,William Burgess Public School,William Burgess School Garden,School Learning Garden,1,11.0,58,None,William Burgess Public School,100 Torrens Avenue,M4J 2P5,None,Toronto-Danforth,14,None,None,None,MULTIPOINT (-79.34816 43.69193)


In [9]:
pollinateto_data['GARDEN_TYPE'] = pollinateto_data['GARDEN_TYPE'].str.split(',', expand=False).str[0]
pollinateto_data['GARDEN_TYPE'].value_counts()

GARDEN_TYPE
School Learning Garden         66
City Park                      24
Communal Garden                20
Spiritual Centre Garden        16
Multiple Front Yard Gardens     9
Boulevard Garden                6
Multi-Residential Garden        4
Multiple Boulevard Gardens      2
Multiple Property Garden        2
Name: count, dtype: int64

In [10]:
pollinateto_data['YEAR_FUNDED'].value_counts()

YEAR_FUNDED
2023    44
2022    40
2020    33
2021    32
Name: count, dtype: int64

In [11]:
pollinateto_data = pollinateto_data.drop(columns=['DESCRIPTION', 'IMAGE_NAME', 'IMAGE_ALT_TEXT'])
pollinateto_data.head()

,_id,ID,YEAR_FUNDED,GROUP_NAME,PROJECT_NAME,GARDEN_TYPE,NUMBER_OF_GARDENS,ESTIMATED_GARDEN_SIZE,NEIGHBOURHOOD_NUMBER,NIA_OR_EN,PRIMARY_LOCATION_NAME,PRIMARY_GARDEN_ADDRESS,POSTAL_CODE,IS_INDIGENOUS_GARDEN,WARD_NAME,WARD_NUMBER,geometry
0,1,1,2020,Appleton Ave. Community Organization,Appleton Ave. Pollinator Corridor,Multiple Front Yard Gardens,18,1250.00,92,None,Private Residential Property,14 Appleton Avenue,M6E 3A4,None,Davenport,9,MULTIPOINT (-79.43796 43.67956)
1,2,2,2020,Centre for Immigrant and Community Services (C...,The ENRICH Pollinator Garden,Communal Garden,4,114.25,128,None,Centre for Immigrant and Community Services,2330 Midland Avenue,M1S 5G5,None,Scarborough-Agincourt,22,MULTIPOINT (-79.27601 43.77839)
2,3,3,2020,Chester School Pollinator Garden Team,Chester School Pollinator Garden,School Learning Garden,1,34.00,57,None,Chester Elementary School,115 Gowan Avenue,M4K 2E4,None,Toronto-Danforth,14,MULTIPOINT (-79.35171 43.6876)
3,4,4,2020,CSPC - Transfiguration,Transfiguration Pollinator Garden,School Learning Garden,1,22.00,7,None,Transfiguration of Our Lord Catholic School,55 Ludstone Drive,M9R 2J2,None,Etobicoke Centre,2,MULTIPOINT (-79.56287 43.69153)
4,5,5,2020,Danforth Gardens Neighbourhood Association,Danforth Gardens Neighbourhood Pollinator Corr...,Multiple Front Yard Gardens,7,28.00,120,None,Private Residential Property,64 Clapperton Avenue,M1L 4K7,None,Scarborough Southwest,20,MULTIPOINT (-79.27635 43.71211)


In [12]:
pollinateto_data['IS_INDIGENOUS_GARDEN'] = pollinateto_data['IS_INDIGENOUS_GARDEN'].replace('None', 'N')
pollinateto_data.head()

,_id,ID,YEAR_FUNDED,GROUP_NAME,PROJECT_NAME,GARDEN_TYPE,NUMBER_OF_GARDENS,ESTIMATED_GARDEN_SIZE,NEIGHBOURHOOD_NUMBER,NIA_OR_EN,PRIMARY_LOCATION_NAME,PRIMARY_GARDEN_ADDRESS,POSTAL_CODE,IS_INDIGENOUS_GARDEN,WARD_NAME,WARD_NUMBER,geometry
0,1,1,2020,Appleton Ave. Community Organization,Appleton Ave. Pollinator Corridor,Multiple Front Yard Gardens,18,1250.00,92,None,Private Residential Property,14 Appleton Avenue,M6E 3A4,N,Davenport,9,MULTIPOINT (-79.43796 43.67956)
1,2,2,2020,Centre for Immigrant and Community Services (C...,The ENRICH Pollinator Garden,Communal Garden,4,114.25,128,None,Centre for Immigrant and Community Services,2330 Midland Avenue,M1S 5G5,N,Scarborough-Agincourt,22,MULTIPOINT (-79.27601 43.77839)
2,3,3,2020,Chester School Pollinator Garden Team,Chester School Pollinator Garden,School Learning Garden,1,34.00,57,None,Chester Elementary School,115 Gowan Avenue,M4K 2E4,N,Toronto-Danforth,14,MULTIPOINT (-79.35171 43.6876)
3,4,4,2020,CSPC - Transfiguration,Transfiguration Pollinator Garden,School Learning Garden,1,22.00,7,None,Transfiguration of Our Lord Catholic School,55 Ludstone Drive,M9R 2J2,N,Etobicoke Centre,2,MULTIPOINT (-79.56287 43.69153)
4,5,5,2020,Danforth Gardens Neighbourhood Association,Danforth Gardens Neighbourhood Pollinator Corr...,Multiple Front Yard Gardens,7,28.00,120,None,Private Residential Property,64 Clapperton Avenue,M1L 4K7,N,Scarborough Southwest,20,MULTIPOINT (-79.27635 43.71211)


In [13]:
# pollinateto_data.to_csv('PollinateTO_cleaned.csv', index=False)

Import Neighborhood Data

In [ ]:

# Define the directory where the CSV files are saved
save_directory = "Cleaned_neighborhood_csvs"

# List of filenames to open
filenames = [
    "geocoded_df.csv",
    "neighbourhoods_df_cleaned.csv",
    "parks_df.csv",
    "schools_df.csv",
    "filtered_gdf.csv"
]

# Dictionary to store the DataFrames
dataframes = {}

# Loop through the filenames and read the CSVs into DataFrames
for filename in filenames:
    file_path = os.path.join(save_directory, filename)
    dataframes[filename] = pd.read_csv(file_path)


# Access the loaded DataFrames by filename (example)
geocoded_df = dataframes.get("geocoded_df.csv")
neighbourhoods_df_cleaned = dataframes.get("neighbourhoods_df_cleaned.csv")
parks_df = dataframes.get("parks_df.csv")
schools_df = dataframes.get("schools_df.csv")
filtered_gdf = dataframes.get("filtered_gdf.csv")



Explore each dataframe

In [15]:
geocoded_df.head()

,STREET_NUM,STREET_NAME,STREET_DIRECTION,POSTAL,WARD_NUMBER,WARD_NAME
0,2681,DANFORTH,,M4C,19,Beaches-East York
1,50,LAING,,M4L,14,Toronto-Danforth
2,23,TORONTO,,M5C,13,Toronto Centre
3,5,JOPLING,S,M9B,3,Etobicoke-Lakeshore
4,0,EXMOOR,,,3,Etobicoke-Lakeshore


In [16]:
neighbourhoods_df_cleaned.head()

,_id,AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,CLASSIFICATION,OBJECTID,geometry
0,1,2502366,174,174,South Eglinton-Davisville,Not an NIA or Emerging Neighbourhood,17824737,MULTIPOLYGON (((-79.3863510515018 43.697831265...
1,2,2502365,173,173,North Toronto,Not an NIA or Emerging Neighbourhood,17824753,MULTIPOLYGON (((-79.3974366551459 43.706929916...
2,3,2502364,172,172,Dovercourt Village,Not an NIA or Emerging Neighbourhood,17824769,MULTIPOLYGON (((-79.4341131654386 43.660145276...
3,4,2502363,171,171,Junction-Wallace Emerson,Not an NIA or Emerging Neighbourhood,17824785,MULTIPOLYGON (((-79.4387000029275 43.667660805...
4,5,2502362,170,170,Yonge-Bay Corridor,Not an NIA or Emerging Neighbourhood,17824801,MULTIPOLYGON (((-79.3840399271321 43.644965094...


In [17]:
parks_df.head()

,_id,LOCATIONID,ASSET_ID,ASSET_NAME,TYPE,AMENITIES,ADDRESS,PHONE,URL,geometry
0,1,298,1126,WILLIAMSON PARK RAVINE,Park,NaN,1680 GERRARD ST E,NaN,https://www.toronto.ca/data/parks/prd/faciliti...,MULTIPOINT (-79.3173566254804 43.6780549678585)
1,2,2342,1328,GLENELLEN PARKETTE,Park,NaN,63 GLENELLEN DR E,NaN,https://www.toronto.ca/data/parks/prd/faciliti...,MULTIPOINT (-79.4954482404427 43.640619013426)
2,3,2281,1317,GATEWAY PARK,Park,NaN,40 EAST LIBERTY ST,NaN,https://www.toronto.ca/data/parks/prd/faciliti...,MULTIPOINT (-79.4119174331757 43.6393331052003)
3,4,1030,964,SMITHWOOD PARK,Park,Playground,79 SMITHWOOD DR,NaN,https://www.toronto.ca/data/parks/prd/faciliti...,MULTIPOINT (-79.5507058668245 43.6408874593971)
4,5,2544,816,PETER STREET BASIN PARK,Park,NaN,370 QUEENS QUAY W,NaN,https://www.toronto.ca/data/parks/prd/faciliti...,MULTIPOINT (-79.3897486591197 43.6384910224043)


In [18]:
schools_df.head()

,_id,OBJECTID,GEO_ID,NAME,BOARD_NAME,SOURCE_ADDRESS,ADDRESS_POINT_ID,ADDRESS_NUMBER,ADDRESS_FULL,POSTAL_CODE,MUNICIPALITY,CITY,PLACE_NAME,CENTRELINE_ID,geometry
0,1,1,330677,A PLUS ACADEMY OF ADVANCEMENT,NaN,2425 EGLINTON AVE E,330677,2425,2425 Eglinton Ave E,M1K 5G8,Scarborough,Toronto,NaN,111254,MULTIPOINT (-79.267103936897 43.7320819460129)
1,2,2,524780,A R S ARMENIAN PRIVATE SCHOOL,NaN,50 HALLCROWN PL,524780,50,50 Hallcrown Pl,M2J 1P7,North York,Toronto,NaN,438652,MULTIPOINT (-79.323156147744 43.770053062482)
2,3,3,20258267,A Y J GLOBAL ACADEMY,NaN,4 LANSING SQ,20258267,4,4 Lansing Sq,M2J 5A2,North York,Toronto,NaN,438287,MULTIPOINT (-79.3252164120635 43.774082570705)
3,4,4,517961,A Y JACKSON SECONDARY SCHOOL,Toronto District School Board,50 FRANCINE DR,517961,50,50 Francine Dr,M2H 2G6,North York,Toronto,A. Y. Jackson Secondary School,436260,MULTIPOINT (-79.3665552934095 43.8052613014219)
4,5,5,13967401,ABACUS MONTESSORI LEARNING CENTRE,NaN,4 CREDIT UNION DR,13967401,4,4 Credit Union Dr,M4A 2N8,North York,Toronto,Latvian Canadian Centre,444179,MULTIPOINT (-79.3190099774403 43.725432232812)


In [19]:
filtered_gdf.head()

,FACILITY,Address,geometry
0,Salvation Army Gateway Shelter,107 Jarvis St,POINT (-79.37247726 43.6523228)
1,Covenant House,20 Gerrard St E,POINT (-79.38143531 43.65950349)
2,Parliament Street Public Library,269 Gerrard St E - 2nd Floor,POINT (-79.36795791 43.66183832)
3,WoodGreen Community Services,650 Queen St E,POINT (-79.35204519 43.65856378)
4,Sheridan Mall,1700 Wilson Ave - # 114,POINT (-79.51253119 43.7209713)


In [20]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(pollinateto_data, geometry='geometry')

# Extract latitude and longitude from the first point in each MultiPoint
gdf['latitude'] = gdf.geometry.apply(lambda geom: geom.geoms[0].y)
gdf['longitude'] = gdf.geometry.apply(lambda geom: geom.geoms[0].x)

# Prepare data for the heatmap (list of [latitude, longitude])
heat_data = gdf[['latitude', 'longitude']].values.tolist()

# Create a base map
m = folium.Map(location=[43.7, -79.4], zoom_start=11)

# Add the heatmap layer
HeatMap(heat_data).add_to(m)

# Save and display the map
m.save("PollinateTO_heatmap.html")

Indigenous Gardens and Neighborhood Data